# Analysis of GEnome-scale Regulatory and Metabolic (GERM) models

MEWpy supports several methods to perform phenotype simulations using GERM models.
The following simulation methods are available in **`mewpy.germ.analysis`**:
- **`FBA`** - requires a Metabolic model
- **`pFBA`** - requires a Metabolic model
- **`RFBA`** - requires a Regulatory-Metabolic model
- **`SRFBA`** - requires a Regulatory-Metabolic model
- **`PROM`** - requires a Regulatory-Metabolic model
- **`CoRegFlux`** - requires a Regulatory-Metabolic model

In addition, **`FBA`** and **`pFBA`** simulation methods are available in the MEWpy **`Simulator`** object.

This example uses the integrated _E. coli_ core model published by [Orth _et al_, 2010](https://doi.org/10.1128/ecosalplus.10.2.1). More information regarding this model is available in `examples.GERM_Models.ipynb` notebook

This example uses the integrated _E. coli_ iMC1010 model published by [Covert _et al_, 2004](https://doi.org/10.1038/nature02456). This model consists of the _E. coli_ iJR904 GEM model published by [Reed _et al_, 2003](https://doi.org/10.1186/gb-2003-4-9-r54) and _E. coli_ iMC1010 TRN published by [Covert _et al_, 2004](https://doi.org/10.1038/nature02456). This model includes 904 metabolic genes, 931 unique biochemical reactions, and a TRN having 1010 regulatory interactions (target-regulators using boolean logic).

This example uses the integrated _M. tuberculosis_ iNJ661 model published by [Chandrasekaran _et al_, 2010](https://doi.org/10.1073/pnas.1005139107). This model consists of the _M. tuberculosis_ iNJ661 GEM model published by [Jamshidi _et al_, 2007](https://doi.org/10.1186/1752-0509-1-26), _M. tuberculosis_ TRN published by [Balazsi _et al_, 2008](https://doi.org/10.1038/msb.2008.63), and gene expression dataset published by [Chandrasekaran _et al_, 2010](https://doi.org/10.1073/pnas.1005139107). This model includes 691 metabolic genes, 1028 unique biochemical reactions, and a TRN having 2018 regulatory interactions (target-regulator).

This example uses the integrated _S. cerevisae_ iMM904 model published by [Banos _et al_, 2017](https://doi.org/10.1186/s12918-017-0507-0). This model consists of the _S. cerevisae_ iMM904 GEM model published by [Mo _et al_, 2009](https://doi.org/10.1186/1752-0509-3-37), _S. cerevisae_ TRN inferred by CoRegNet published by [Nicolle _et al_, 2015](https://doi.org/10.1093/bioinformatics/btv305), and gene expression datasets published by [Brauer _et al_, 2005](https://doi.org/10.1091/mbc.e04-11-0968) and [DeRisi _et al_, 1997](https://doi.org/10.1126/science.278.5338.680). This model includes 904 metabolic genes, 1557 unique biochemical reactions, and a TRN having 3748 regulatory interactions (target-regulators separated in co-activators and co-repressors).

In [1]:
# imports
import os
from pathlib import Path

from mewpy.io import Engines, Reader, read_model
from mewpy.germ.analysis import *

In [2]:
# readers
path = Path(os.getcwd()).joinpath('models', 'germ')

# E. coli core
core_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('e_coli_core.xml'))
core_trn_reader = Reader(Engines.BooleanRegulatoryCSV,
                         path.joinpath('e_coli_core_trn.csv'),
                         sep=',',
                         id_col=0,
                         rule_col=2,
                         aliases_cols=[1],
                         header=0)

# E. coli iMC1010
imc1010_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('iJR904.xml'))
imc1010_trn_reader = Reader(Engines.BooleanRegulatoryCSV,
                            path.joinpath('iMC1010.csv'),
                            sep=',',
                            id_col=0,
                            rule_col=4,
                            aliases_cols=[1, 2, 3],
                            header=0)

# M. tuberculosis iNJ661
inj661_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('iNJ661.xml'))
inj661_trn_reader = Reader(Engines.TargetRegulatorRegulatoryCSV,
                           path.joinpath('iNJ661_trn.csv'),
                           sep=';',
                           target_col=0,
                           regulator_col=1,
                           header=None)
inj661_gene_expression_path = path.joinpath('iNJ661_gene_expression.csv')

# S. cerevisae iMM904
imm904_gem_reader = Reader(Engines.MetabolicSBML, path.joinpath('iMM904.xml'))
imm904_trn_reader = Reader(Engines.CoExpressionRegulatoryCSV,
                           path.joinpath('iMM904_trn.csv'),
                           sep=',',
                           target_col=2,
                           co_activating_col=3,
                           co_repressing_col=4,
                           header=0)

## Working with GERM model analysis
In the `mewpy.germ.analysis` package, simulation methods are derived from a **`LinearProblem`** object having the following attributes and methods:
- `method` - the name of the simulation method
- `model` - the model used to build the linear problem
- `solver` - a MEWpy solver instance having the linear programming implementation of variables and constraints in the selected solver. The following solvers are available: _CPLEX_; _GUROBI_; _OPTLANG_
- `constraints` - The representation of ODE to be implemented in the solver instance using linear programming
- `variables` - The representation of the system variables to be implemented in the solver instance using linear programming
- `objective` - A linear representation of the objective function associated with the linear problem

A simulation method includes two important methods:
- **`build`** - the build method is responsible for retrieving variables and constraints from a GERM model according to the mathematical formulation of each simulation method
- **`optimize`** - the optimize method is responsible for solving the linear problem using linear programming or mixed-integer linear programming. This method accepts method-specific arguments (initial state, dynamic, etc) and solver-specific arguments (linear, minimize, constraints, get_values, etc). These arguments can override temporarily some constraints or variables during the optimization.

In [3]:
# showcase of a simulation method

# reading the E. coli core model
model = read_model(core_gem_reader, core_trn_reader)

# initialization does not build the model automatically
srfba = SRFBA(model).build()
srfba

Method,SRFBA
Model,Model e_coli_core - E. coli core model - Orth et al 2010
Variables,486
Constraints,326
Objective,{'Biomass_Ecoli_core': 1.0}
Solver,CplexSolver
Synchronized,True


The `optimize` interface creates a `ModelSolution` output by default containing the objective value, value of each variable in the solution, among others. Alternatively, `optimize` can create a simple solver `Solution` object.

In [4]:
# optimization creates a ModelSolution object by default
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model e_coli_core - E. coli core model - Orth et al 2010
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684829
Status,optimal


One can generate a pandas `DataFrame` using the **`to_frame()`** method of the **`ModelSolution`** object.
This data frame contains the obtained expression coefficients for the regulatory environmental stimuli linked to the metabolic model and exchange fluxes.

In [5]:
# a solution can be converted into a df
solution.to_frame()

regulatory                                    \
            regulatory variable variable type minimum coefficient   
EX_ac_e                     NaN           NaN                 NaN   
EX_acald_e                  NaN           NaN                 NaN   
EX_akg_e                    NaN           NaN                 NaN   
EX_co2_e                    NaN           NaN                 NaN   
EX_etoh_e                   NaN           NaN                 NaN   
EX_for_e                    NaN           NaN                 NaN   
EX_fru_e                    NaN           NaN                 NaN   
EX_fum_e                    NaN           NaN                 NaN   
EX_glc__D_e                 NaN           NaN                 NaN   
EX_gln__L_e                 NaN           NaN                 NaN   
EX_glu__L_e                 NaN           NaN                 NaN   
EX_h_e                      NaN           NaN                 NaN   
EX_h2o_e                    NaN           NaN                 NaN   
EX_lac__D_e                 NaN           NaN                 NaN   
EX_mal__L_e                 NaN           NaN                 NaN   
EX_nh4_e                    NaN           NaN                 NaN   
EX_o2_e                     NaN           NaN                 NaN   
EX_pi_e                     NaN           NaN                 NaN   
EX_pyr_e                    NaN           NaN                 NaN   
EX_succ_e                   NaN           NaN                 NaN   

                                                          metabolic  \
            maximum coefficient expression coefficient     exchange   
EX_ac_e                     NaN                    NaN      EX_ac_e   
EX_acald_e                  NaN                    NaN   EX_acald_e   
EX_akg_e                    NaN                    NaN     EX_akg_e   
EX_co2_e                    NaN                    NaN     EX_co2_e   
EX_etoh_e                   NaN                    NaN    EX_etoh_e   
EX_for_e                    NaN                    NaN     EX_for_e   
EX_fru_e                    NaN                    NaN     EX_fru_e   
EX_fum_e                    NaN                    NaN     EX_fum_e   
EX_glc__D_e                 NaN                    NaN  EX_glc__D_e   
EX_gln__L_e                 NaN                    NaN  EX_gln__L_e   
EX_glu__L_e                 NaN                    NaN  EX_glu__L_e   
EX_h_e                      NaN                    NaN       EX_h_e   
EX_h2o_e                    NaN                    NaN     EX_h2o_e   
EX_lac__D_e                 NaN                    NaN  EX_lac__D_e   
EX_mal__L_e                 NaN                    NaN  EX_mal__L_e   
EX_nh4_e                    NaN                    NaN     EX_nh4_e   
EX_o2_e                     NaN                    NaN      EX_o2_e   
EX_pi_e                     NaN                    NaN      EX_pi_e   
EX_pyr_e                    NaN                    NaN     EX_pyr_e   
EX_succ_e                   NaN                    NaN    EX_succ_e   

                                                                            
            variable type metabolite lower bound upper bound          flux  
EX_ac_e          reaction       ac_e         0.0      1000.0 -2.273737e-13  
EX_acald_e       reaction    acald_e         0.0      1000.0 -2.273737e-13  
EX_akg_e         reaction      akg_e         0.0      1000.0 -2.273737e-13  
EX_co2_e         reaction      co2_e     -1000.0      1000.0  2.280983e+01  
EX_etoh_e        reaction     etoh_e         0.0      1000.0 -2.273737e-13  
EX_for_e         reaction      for_e         0.0      1000.0  0.000000e+00  
EX_fru_e         reaction      fru_e         0.0      1000.0  0.000000e+00  
EX_fum_e         reaction      fum_e         0.0      1000.0  0.000000e+00  
EX_glc__D_e      reaction   glc__D_e       -10.0      1000.0 -1.000000e+01  
EX_gln__L_e      reaction   gln__L_e         0.0      1000.0  0.000000e+00  
EX_glu__L_e    

One can generate a **`Summary`** object using the **`to_summary()`** method of the **`ModelSolution`** object.
This summary contains the following data:
- `inputs` - regulatory and metabolic inputs for the simulation method
- `outputs` - regulatory and metabolic inputs for the simulation method
- `metabolic` - values of the metabolic variables
- `regulatory` - values of the regulatory variables
- `objective` - the objective value
- `df` - the summary of inputs and outputs in the regulatory and metabolic layers

In [6]:
# a solution can be converted into a summary solution
summary = solution.to_summary()
summary

In [7]:
# inputs + outputs of the metabolic-regulatory variables
summary.df

regulatory                             \
         regulatory variable      variable type    role   
b0008                  b0008       target, gene  output   
b0080                  b0080  regulator, target  output   
b0113                  b0113  regulator, target  output   
b0114                  b0114       target, gene  output   
b0115                  b0115       target, gene  output   
...                      ...                ...     ...   
EX_h_e                   NaN                NaN     NaN   
EX_h2o_e                 NaN                NaN     NaN   
EX_nh4_e                 NaN                NaN     NaN   
EX_o2_e                  NaN                NaN     NaN   
EX_pi_e                  NaN                NaN     NaN   

                                metabolic                                   \
         expression coefficient  reaction variable type metabolite    role   
b0008                       1.0       NaN           NaN        NaN     NaN   
b0080                       1.0       NaN           NaN        NaN     NaN   
b0113                       1.0       NaN           NaN        NaN     NaN   
b0114                       1.0       NaN           NaN        NaN     NaN   
b0115                       1.0       NaN           NaN        NaN     NaN   
...                         ...       ...           ...        ...     ...   
EX_h_e                      NaN    EX_h_e      reaction        h_e  output   
EX_h2o_e                    NaN  EX_h2o_e      reaction      h2o_e  output   
EX_nh4_e                    NaN  EX_nh4_e      reaction      nh4_e   input   
EX_o2_e                     NaN   EX_o2_e      reaction       o2_e   input   
EX_pi_e                     NaN   EX_pi_e      reaction       pi_e   input   

                     
               flux  
b0008           NaN  
b0080           NaN  
b0113           NaN  
b0114           NaN  
b0115           NaN  
...             ...  
EX_h_e    17.530865  
EX_h2o_e  29.175827  
EX_nh4_e  -4.765319  
EX_o2_e  -21.799493  
EX_pi_e   -3.214895  

[166 rows x 9 columns]

In [8]:
# values of the metabolic variables
summary.metabolic

,reaction,variable type,metabolite,role,flux
EX_co2_e,EX_co2_e,reaction,co2_e,output,22.809833
EX_glc__D_e,EX_glc__D_e,reaction,glc__D_e,input,-10.000000
EX_h_e,EX_h_e,reaction,h_e,output,17.530865
EX_h2o_e,EX_h2o_e,reaction,h2o_e,output,29.175827
EX_nh4_e,EX_nh4_e,reaction,nh4_e,input,-4.765319
EX_o2_e,EX_o2_e,reaction,o2_e,input,-21.799493
EX_pi_e,EX_pi_e,reaction,pi_e,input,-3.214895


In [9]:
# values of the regulatory variables
summary.regulatory

,regulatory variable,variable type,role,expression coefficient
b0008,b0008,"target, gene",output,1.0
b0080,b0080,"regulator, target",output,1.0
b0113,b0113,"regulator, target",output,1.0
b0114,b0114,"target, gene",output,1.0
b0115,b0115,"target, gene",output,1.0
...,...,...,...,...
CRPnoGLM,CRPnoGLM,"regulator, target",output,1.0
NRI_hi,NRI_hi,"regulator, target",output,0.0
NRI_low,NRI_low,"regulator, target",output,-0.0
surplusFDP,surplusFDP,"regulator, target",output,0.0


In [10]:
# objective value
summary.objective

,value,direction
Biomass_Ecoli_core,0.873922,maximize


In [11]:
# values of the metabolic and regulatory inputs
summary.inputs

regulatory                                         metabolic  \
             regulator variable type expression coefficient     reaction   
EX_glc__D_e        NaN           NaN                    NaN  EX_glc__D_e   
EX_nh4_e           NaN           NaN                    NaN     EX_nh4_e   
EX_o2_e            NaN           NaN                    NaN      EX_o2_e   
EX_pi_e            NaN           NaN                    NaN      EX_pi_e   

                                                 
            variable type metabolite       flux  
EX_glc__D_e      reaction   glc__D_e -10.000000  
EX_nh4_e         reaction      nh4_e  -4.765319  
EX_o2_e          reaction       o2_e -21.799493  
EX_pi_e          reaction       pi_e  -3.214895

In [12]:
# values of the metabolic and regulatory outputs
summary.outputs

regulatory                                           metabolic  \
                target      variable type expression coefficient  reaction   
b0008            b0008       target, gene                    1.0       NaN   
b0080            b0080  regulator, target                    1.0       NaN   
b0113            b0113  regulator, target                    1.0       NaN   
b0114            b0114       target, gene                    1.0       NaN   
b0115            b0115       target, gene                    1.0       NaN   
...                ...                ...                    ...       ...   
surplusFDP  surplusFDP  regulator, target                    0.0       NaN   
surplusPYR  surplusPYR  regulator, target                    0.0       NaN   
EX_co2_e           NaN                NaN                    NaN  EX_co2_e   
EX_h_e             NaN                NaN                    NaN    EX_h_e   
EX_h2o_e           NaN                NaN                    NaN  EX_h2o_e   

                                                
           variable type metabolite       flux  
b0008                NaN        NaN        NaN  
b0080                NaN        NaN        NaN  
b0113                NaN        NaN        NaN  
b0114                NaN        NaN        NaN  
b0115                NaN        NaN        NaN  
...                  ...        ...        ...  
surplusFDP           NaN        NaN        NaN  
surplusPYR           NaN        NaN        NaN  
EX_co2_e        reaction      co2_e  22.809833  
EX_h_e          reaction        h_e  17.530865  
EX_h2o_e        reaction      h2o_e  29.175827  

[162 rows x 7 columns]

## GERM model and phenotype simulation workflow
A phenotype simulation method must be initialized with a GERM model. A common workflow to work with GERM models and simulation methods is suggested as follows:
1. `model = read_model(reader1, reader2)` - read the model
2. `rfba = RFBA(model)` - initialize the simulation method
3. `rfba.build()` - build the linear problem
4. `solution = rfba.optimize()` - perform the optimization
5. `model.reactions['MY_REACTION'].bounds = (0, 0)` - make changes to the model
6. `solution = RFBA(model).build().optimize()` - initialize, build and optimize the simulation method

In this workflow, _model_ and _rfba_ instances are not connected with each other. Future rfba's optimization will generate the same output even if we make changes to the model. That is, _model_ and _rfba_ are not synchronized and attached to each other.
<br>

Although building linear problems is considerably fast for most models, there is a second workflow to work with GERM models and simulation methods:
1. `model = read_model(reader1, reader2)` - read the model
2. `rfba = RFBA(model, attach=True)` - initialize the simulation method and attach it to the model
3. `rfba.build()` - build the linear problem
4. `solution = rfba.optimize()` - perform the optimization
5. `model.reactions['MY_REACTION'].bounds = (0, 0)` - make changes to the model
6. `rxn_ko_solution = rfba.optimize()` - perform the optimization again but this time with the reaction deletion

In [13]:
# read, build, optimize
model = read_model(core_gem_reader, core_trn_reader)
srfba = SRFBA(model).build()
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model e_coli_core - E. coli core model - Orth et al 2010
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684829
Status,optimal


In [14]:
# make changes and then build, optimize
model.regulators['b3261'].ko()
srfba = SRFBA(model).build()
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model e_coli_core - E. coli core model - Orth et al 2010
Objective,Biomass_Ecoli_core
Objective value,0.0
Status,optimal


In [15]:
# second workflow
model = read_model(core_gem_reader, core_trn_reader)
srfba = SRFBA(model, attach=True).build()
solution = srfba.optimize()
print('Wild-type growth rate', solution.objective_value)

# applying the knockout
model.regulators['b3261'].ko()
solution = srfba.optimize()
print('KO growth rate', solution.objective_value)

Wild-type growth rate 0.8739215069684829
KO growth rate 0.0


In addition, one can attach as many simulation methods as needed to a single model instance. This behavior eases the comparison between simulation methods

In [16]:
# many simulation methods attached
model = read_model(core_gem_reader, core_trn_reader)
fba = FBA(model, attach=True).build()
pfba = pFBA(model, attach=True).build()
rfba = RFBA(model, attach=True).build()
srfba = SRFBA(model, attach=True).build()

# applying the knockout
model.regulators['b3261'].ko()

print('FBA KO growth rate:', fba.optimize().objective_value)
print('pFBA KO sum of fluxes:', pfba.optimize().objective_value)
print('RFBA KO growth rate:', rfba.optimize().objective_value)
print('SRFBA KO growth rate:', srfba.optimize().objective_value)
print()

# restore the model
model.undo()
print('FBA WT growth rate:', fba.optimize().objective_value)
print('pFBA WT sum of fluxes:', pfba.optimize().objective_value)
print('RFBA WT growth rate:', rfba.optimize().objective_value)
print('SRFBA WT growth rate:', srfba.optimize().objective_value)

FBA KO growth rate: 0.8739215069684303
pFBA KO sum of fluxes: 93768.8478640836
RFBA KO growth rate: 0.8513885233462081
SRFBA KO growth rate: 0.0

FBA WT growth rate: 0.8739215069684303
pFBA WT sum of fluxes: 93768.8478640836
RFBA WT growth rate: 0.8513885233462081
SRFBA WT growth rate: 0.8739215069684829


## FBA and pFBA

MEWpy supports **`FBA`** and **`pFBA`** simulation methods using GERM models.
<br>
**`FBA`** is a phenotype simulation method based on mass balance constraints retrieved from metabolites and reactions found in a GEM model. FBA is aimed at finding the maximum value for the objective function. As the biomass reaction is often used as objective function, FBA is often used to find the optimal growth rate of an organism. For more details consult: [https://doi.org/10.1038/nbt.1614](https://doi.org/10.1038/nbt.1614). In addition, **`mewpy.germ.analysis.FBA`** also takes into consideration the coefficients of metabolic genes, thus limiting reactions bounds to the corresponding gene states.
<br>
**`pFBA`** is a phenotype simulation method based on **`FBA`**, as this method also finds the optimal growth rate. However, the objective function of pFBA consists of minimizing the total sum of all fluxes, and thus finding the subset of genes and proteins that may contribute to the most efficient metabolic network topology [Lewis _et al_, 2010](https://doi.org/10.1038/msb.2010.47).
<br>
**`FBA`** and **`pFBA`** are both available in the **`mewpy.germ.analysis`** package. Alternatively, one can use the simple and optimized versions **`slim_fba`** and **`slim_pfba`**. Likewise, **`FBA`** and **`pFBA`** are available in MEWpy's **`Simulator`**, which is the common interface to perform simulations using GERM models, COBRApy models, and Reframed models.

In [17]:
# using FBA analysis
met_model = read_model(core_gem_reader)
FBA(met_model).build().optimize()

Method,FBA
Model,Model e_coli_core - E. coli core model - Orth et al 2010
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684303
Status,optimal


In [18]:
# using slim FBA analysis
slim_fba(met_model)

0.8739215069684303

In [19]:
# using MEWpy simulator
from mewpy.simulation import get_simulator
simulator = get_simulator(met_model)
simulator.simulate()

objective: 0.8739215069684303
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.FBA

In [20]:
from mewpy.simulation import SimulationMethod

# pfba version
print(pFBA(met_model).build().optimize().objective_value)
print(slim_pfba(met_model))
print(simulator.simulate(method=SimulationMethod.pFBA))

93768.8478640836
93768.8478640836
objective: 0.8739215069684303
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.pFBA


## FVA and Deletions
The **`mewpy.germ.analysis`** package includes the **`FVA`** method to inspect the solution space of a GEM model.
FVA computes the minimum and maximum possible fluxes of each reaction in a metabolic model. This method can be used to identify reactions limiting cellular growth. This method return a pandas `DataFrame` with the minium and maximum fluxes (columns) for each reaction (index).
<br>
The `mewpy.germ.analysis` package includes **`single_gene_deletion`** and **`single_reaction_deletion`** methods to inspect _in silico_ genetic strategies. These methods perform an FBA phenotype simulation of a single reaction deletion or gene knockout for all reactions and genes in the metabolic model. These methods are faster than iterating through the model reactions or genes using the `ko()` method.

In [21]:
# FVA returns the DataFrame with minium and maximum values of each reaction
fva(met_model)

,minimum,maximum
ACALD,-7.105427e-15,0.000000
ACALDt,0.000000e+00,0.000000
ACKr,0.000000e+00,0.000000
ACONTa,6.007250e+00,6.007250
ACONTb,6.007250e+00,6.007250
...,...,...
TALA,1.496984e+00,1.496984
THD2,0.000000e+00,0.000000
TKT1,1.496984e+00,1.496984
TKT2,1.181498e+00,1.181498


In [22]:
# FVA returns the DataFrame with minium and maximum values of each reaction
fva(met_model)

,minimum,maximum
ACALD,-7.105427e-15,0.000000
ACALDt,0.000000e+00,0.000000
ACKr,0.000000e+00,0.000000
ACONTa,6.007250e+00,6.007250
ACONTb,6.007250e+00,6.007250
...,...,...
TALA,1.496984e+00,1.496984
THD2,0.000000e+00,0.000000
TKT1,1.496984e+00,1.496984
TKT2,1.181498e+00,1.181498


In [23]:
# single reaction deletion
single_reaction_deletion(met_model)

,growth,status
ACALD,0.873922,Optimal
ACALDt,0.873922,Optimal
ACKr,0.873922,Optimal
ACONTa,0.000000,Optimal
ACONTb,0.000000,Optimal
...,...,...
TALA,0.864759,Optimal
THD2,0.873922,Optimal
TKT1,0.864759,Optimal
TKT2,0.866674,Optimal


In [24]:
# single gene deletion
single_gene_deletion(met_model)

,growth,status
b0351,0.873922,Optimal
b1241,0.873922,Optimal
s0001,0.211141,Optimal
b2296,0.873922,Optimal
b3115,0.873922,Optimal
...,...,...
b2464,0.873922,Optimal
b0008,0.873922,Optimal
b2935,0.873922,Optimal
b2465,0.873922,Optimal


In [25]:
# single gene deletion for specific genes
single_gene_deletion(met_model, genes=met_model.reactions['ACONTa'].genes)

,growth,status
b0118,0.873922,Optimal
b1276,0.873922,Optimal


## Regulatory Truth Table
The regulatory truth table of a regulatory model contains the evaluation of all regulatory interactions.
The **`mewpy.germ.analysis.regulatory_truth_table`** method creates the combination between the regulators and target genes given a regulatory model. This function returns a pandas `DataFrame` having the regulators' values in the columns and targets' outcome in the index.

In [26]:
# regulatory truth table for the regulatory model
reg_model = read_model(core_trn_reader)
regulatory_truth_table(reg_model)

,result,surplusFDP,surplusPYR,b0113,b3261,b0400,pi_e,b4401,b1334,b3357,...,TALA,PGI,fru_e,ME2,ME1,GLCpts,PYK,PFK,LDH_D,SUCCt2_2
b0008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0080,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0113,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0114,1,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0115,1,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRPnoGLM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NRI_hi,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NRI_low,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surplusFDP,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## RFBA
**`RFBA`** is a phenotype simulation method based on the integration of a GEM model with a TRN at the genome-scale. The TRN consists of a set of regulatory interactions formulated with boolean and propositional logic. The TRN contains a boolean algebra expression for each target gene. This boolean rule determines whether the target gene is active (1) or not (0) according to the state of the regulators (active or inactive). Then, the TRN is integrated with the GEM model using the reactions' GPR rules. It is also common to find metabolites and reactions as regulators/environmental stimuli in the TRN, completing the integration with the GEM model.

In **`RFBA`**, a synchronous evaluation of all regulatory interactions in the regulatory model is performed first. This first simulation is used to retrieve the regulatory state (regulators' coefficients). Then, the regulatory state is translated into a metabolic state (metabolic genes' coefficients) by performing another synchronous evaluation of all regulatory interactions in the regulatory model. Finally, the resulting metabolic state is used to decode the constraints imposed by the regulatory model upon evaluation of the reactions' GPRs with the targets' state.

**`RFBA`** supports steady-state or dynamic phenotype simulations. Dynamic **`RFBA`** simulation performs sequential optimizations while the regulatory state is updated each time using the reactions and metabolites coefficients of the previous optimization. Dynamic **`RFBA`** simulation stops when two identical solutions are found.

**`RFBA`** is available in the **`mewpy.germ.analysis`** package. Alternatively, one can use the simple and optimized version **`slim_rfba`**.

For more details consult: [https://doi.org/10.1038/nature02456](https://doi.org/10.1038/nature02456).

For this example we will be using _E. coli_ iMC1010 model available at _models/regulation/iJR904_srfba.xml_ and _models/regulation/iMC1010.csv_

In [27]:
# loading model
model = read_model(imc1010_gem_reader, imc1010_trn_reader)

# objective function
BIOMASS_ID = 'BiomassEcoli'
model.objective = {BIOMASS_ID: 1}
model

Model,iJR904
Name,Reed2003 - Genome-scale metabolic network of Escherichia coli (iJR904)
Types,"regulatory, metabolic"
Compartments,"e, c"
Reactions,1083
Metabolites,768
Genes,904
Exchanges,150
Demands,0
Sinks,0
Objective,BiomassEcoli


**`RFBA`** can be simulated using an initial regulatory state. This initial state will be considered during the synchronous evaluation of all regulatory interactions in the regulatory model and determine the metabolic state. The set-up of the regulators' initial state in integrated models is a difficult task. Most of the time, the initial state is not known and hinders feasible solutions during simulation. If the initial state is not provided to RFBA, this method will consider that all regulators are active. However, this initial state is clearly not the best, as many essential reactions can be switched off.
<br>
To relax some constraints, the initial state of a regulatory metabolite is inferred from its exchange reaction, namely the absolute value of the lower bound. Likewise, the initial state of a regulatory reaction is inferred from its upper bound. Even so, this initial state is likely to yield infeasible solutions.
<br>
### Find conflicts
To mitigate these conflicts between the regulatory and metabolic state, one can use the **`mewpy.germ.analysis.find_conflicts()`** method to ease the set-up of the initial state. This method can be used to find regulatory states that affect the growth of the cell. It tries to find the regulatory states that lead to knockouts of essential genes and deletion of essential reactions.
Note that, **`find_conflicts()`** results should be carefully analyzed, as this method does not detect indirect conflicts. Please consult the method for more details and the example bellow.

In [28]:
# we can see that 3 regulators are affecting the following essential genes: b2574; b1092; b3730
repressed_genes, repressed_reactions = find_conflicts(model)
repressed_genes

,interaction,b0676,Stringent,b4390
b3730,b3730 || 1 = b0676,0.0,NaN,NaN
b1092,b1092 || 1 = ( ~ Stringent),NaN,1.0,NaN
b2574,b2574 || 1 = ( ~ b4390),NaN,NaN,1.0


**`find_conflicts()`** suggests that three essential genes (_b2574_; _b1092_; _b3730_) are being affected by three regulators (_b4390_, _Stringent_, _b0676_). However, some regulators do not affect growth directly, as they are being regulated by other regulators, environmental stimuli, metabolites and reactions.

In [29]:
# regulator-target b4390 is active in high-NAD conditions (environmental stimuli)
model.get('b4390')

Identifier,b4390
Name,b4390
Aliases,"NadR, nadr, nadR, b4390"
Model,iJR904
Types,"regulator, target"
Coefficients,"(0.0, 1.0)"
Active,True
Interactions,"b0931_interaction, b2574_interaction"
Targets,"b0931, b2574"
Environmental stimulus,False
Interaction,b4390 || 1 = high-NAD


In [30]:
# regulator-target b0676 is active if both acgam metabolite and AGDC reaction are inactive (cannot carry flux)
model.get('b0676')

Identifier,b0676
Name,b0676
Aliases,"nagC, nagc, NagC, b0676"
Model,iJR904
Types,"regulator, target"
Coefficients,"(0.0, 1.0)"
Active,True
Interactions,"b0677_interaction, b0678_interaction, b0679_interaction, b3730_interaction"
Targets,"b0677, b0678, b0679, b3730"
Environmental stimulus,False
Interaction,b0676 || 1 = ( ~ ((acgam_e > 0) | (AGDC > 0)))


In [31]:
# initial state inferred from the find_conflicts method.
initial_state = {
    'Stringent': 0.0,
    'high-NAD': 0.0,
    'AGDC': 0.0,
}

# steady-state RFBA
rfba = RFBA(model).build()
solution = rfba.optimize(initial_state=initial_state)
solution

Method,RFBA
Model,Model iJR904 - Reed2003 - Genome-scale metabolic network of Escherichia coli (iJR904)
Objective,BiomassEcoli
Objective value,0.8517832811766191
Status,optimal


In [32]:
# using the slim version
slim_rfba(model, initial_state=initial_state)

0.8517832811766191

In [33]:
# dynamic RFBA
dynamic_solution = rfba.optimize(initial_state=initial_state, dynamic=True)
dynamic_solution.solutions

{'t_0': RFBA Solution
  Objective value: 0.8517832812011552
  Status: optimal,
 't_1': RFBA Solution
  Objective value: 0.5898746223794296
  Status: optimal,
 't_2': RFBA Solution
  Objective value: 0.5476893156457431
  Status: optimal,
 't_3': RFBA Solution
  Objective value: 0.5476893156724856
  Status: optimal,
 't_4': RFBA Solution
  Objective value: 0.5476893156724856
  Status: optimal}

## SRFBA
**`SRFBA`** is a phenotype simulation method based on the integration of a GEM model with a TRN at the genome-scale. The TRN consists of a set of regulatory interactions formulated with boolean and propositional logic. The TRN contains a boolean algebra expression for each target gene. This boolean rule determines whether the target gene is active (1) or not (0) according to the state of the regulators (active or inactive). Then, the TRN is integrated with the GEM model using the reactions' GPR rules. It is also common to find metabolites and reactions as regulators/environmental stimuli in the TRN, completing the integration with the GEM model.

**`SRFBA`** performs a single steady-state simulation using both metabolic and regulatory constraints found in the integrated model. This method uses Mixed-Integer Linear Programming to solve nested boolean algebra expressions formulated from the structure of the regulatory layer (regulatory interactions) and metabolic layer (GPR rules). Hence, this method adds auxiliary variables representing intermediate boolean variables and operators. Finally, the linear problem also includes a boolean variable and constraint for each reaction linking the outcome of the interactions and GPR constraints to the mass balance constraints.

**`SRFBA`** only supports steady-state simulations.

**`SRFBA`** is available in the **`mewpy.germ.analysis`** package. Alternatively, one can use the simple and optimized version **`slim_srfba`**.

For more details consult: [https://doi.org/10.1038%2Fmsb4100141](https://doi.org/10.1038%2Fmsb4100141).

For this example we will be using _E. coli_ iMC1010 model available at _models/regulation/iJR904_srfba.xml_ and _models/regulation/iMC1010.csv_

In [34]:
# loading model
model = read_model(imc1010_gem_reader, imc1010_trn_reader)

# objective function
BIOMASS_ID = 'BiomassEcoli'
model.objective = {BIOMASS_ID: 1}
model

Model,iJR904
Name,Reed2003 - Genome-scale metabolic network of Escherichia coli (iJR904)
Types,"regulatory, metabolic"
Compartments,"e, c"
Reactions,1083
Metabolites,768
Genes,904
Exchanges,150
Demands,0
Sinks,0
Objective,BiomassEcoli


**`SRFBA`** does not need an initial state in most cases, as this method performs a steady-state simulation using MILP. The solver tries to find the regulatory state favoring reactions that contribute to faster growth rates. Accordingly, regulatory variables can take values between zero and one.

In [35]:
# steady-state SRFBA
srfba = SRFBA(model).build()
solution = srfba.optimize()
solution

Method,SRFBA
Model,Model iJR904 - Reed2003 - Genome-scale metabolic network of Escherichia coli (iJR904)
Objective,BiomassEcoli
Objective value,0.8218562181811009
Status,optimal


In [36]:
# using the slim version
slim_srfba(model)

0.8218562181811009

## iFVA and iDeletions
The `mewpy.germ.analysis` package includes an integrated version of the **`FVA`** method named **`iFVA`**. This method can be used to inspect the solution space of an integrated GERM model.
**`iFVA`** computes the minimum and maximum possible fluxes of each reaction in a metabolic model using one of the integrated analysis mentioned above (**`RFBA`** or **`SRFBA`**). This method return a pandas `DataFrame` with the minium and maximum fluxes (columns) for each reaction (index).
<br>
The `mewpy.germ.analysis` package also includes **`isingle_gene_deletion`**, **`isingle_reaction_deletion`**, and **`isingle_regulator_deletion`** methods to inspect _in silico_ genetic strategies in integrated GERM models.

In [37]:
# loading model
model = read_model(imc1010_gem_reader, imc1010_trn_reader)

# objective function
BIOMASS_ID = 'BiomassEcoli'
model.objective = {BIOMASS_ID: 1}
model

Model,iJR904
Name,Reed2003 - Genome-scale metabolic network of Escherichia coli (iJR904)
Types,"regulatory, metabolic"
Compartments,"e, c"
Reactions,1083
Metabolites,768
Genes,904
Exchanges,150
Demands,0
Sinks,0
Objective,BiomassEcoli


In [38]:
# iFVA of the first fifteen reactions using srfba (the default method). Fraction inferior to 1 (default) to relax the constraints
reactions_ids = list(model.reactions)[:15]
ifva(model, fraction=0.9, reactions=reactions_ids, method='srfba')

,minimum,maximum
12PPDt,-2.328306e-10,0.000000
2DGLCNRx,0.000000e+00,0.000000
2DGLCNRy,0.000000e+00,0.000000
2DGULRx,0.000000e+00,0.000000
2DGULRy,0.000000e+00,0.000000
3HCINNMH,0.000000e+00,0.000000
3HPPPNH,0.000000e+00,0.000000
4HTHRS,0.000000e+00,0.000000
5DGLCNR,-1.344363e+00,0.000000
A5PISO,3.106617e-02,0.034518


## PROM
**`PROM`** is a probabilistic-based phenotype simulation method for integrated models. This method circumvents discrete constraints created by **`RFBA`** and **`SRFBA`**. This method uses a continuous approach: reactions' constraints are proportional to the probabilities of related genes being active. The probability of an active metabolic gene is inferred from the TRN and gene expression dataset. In detail, gene probability is calculated according to the number of samples that the gene is active when its regulator is inactive.

**`PROM`** performs a single steady-state simulation using the probabilistic-based constraints to limit flux through some reactions. This method cannot perform wild-type phenotype simulations though, as probabilities are calculated for single regulator deletion. Hence, this method is adequate to predict the effect of regulator perturbations.

**`PROM`** can generate a **`KOSolution`** containing the solution of each regulator knock-out.

**`PROM`** is available in the **`mewpy.germ.analysis`** package. Alternatively, one can use the simple and optimized version **`slim_prom`**.

For more details consult: [https://doi.org/10.1073/pnas.1005139107](https://doi.org/10.1073/pnas.1005139107).

For this example we will be using _M. tuberculosis_ iNJ661 model available at _models/regulation/iNJ661.xml_, _models/regulation/iNJ661_trn.csv_, and _iNJ661_gene_expression.csv_.

In [39]:
# loading model
model = read_model(inj661_gem_reader, inj661_trn_reader)

# objective function
BIOMASS_ID = 'biomass_Mtb_9_60atp_test_NOF'
model.objective = {BIOMASS_ID: 1}
model

Model,iNJ661
Name,M. tuberculosis iNJ661 model - Jamshidi et al 2007
Types,"regulatory, metabolic"
Compartments,"c, e"
Reactions,1028
Metabolites,828
Genes,661
Exchanges,88
Demands,0
Sinks,0
Objective,biomass_Mtb_9_60atp_test_NOF


**`PROM`** phenotype simulation requires an initial state that must be inferred from the TRN and gene expression dataset.
Besides, the format of the initial state is slightly different from **`RFBA`** and **`SRFBA`** initial states. **`PROM`**'s initial state must be a dictionary in the following format:
- keys -> tuple of regulator and target gene identifiers
- value -> probability of this regulatory interaction inferred from the gene expression dataset

<br>

**`mewpy.omics`** package contains the required methods to perform a quantile preprocessing of the gene expression dataset. Then, one can use the `mewpy.germ.analysis.prom.target_regulator_interaction_probability()` method to infer **`PROM`**'s initial state


In [40]:
# computing PROM target-regulator interaction probabilities using quantile preprocessing pipeline
from mewpy.omics import ExpressionSet

expression = ExpressionSet.from_csv(file_path=inj661_gene_expression_path, sep=';', index_col=0, header=None)
quantile_expression, binary_expression = expression.quantile_pipeline()
initial_state, _ = target_regulator_interaction_probability(model,
                                                            expression=quantile_expression,
                                                            binary_expression=binary_expression)
initial_state

{('Rv3396c', 'Rv0001'): 1,
 ('Rv3396c', 'Rv3575c'): 0.5075528700906344,
 ('Rv3396c', 'Rv3676'): 1,
 ('Rv3411c', 'Rv0001'): 0.5421686746987951,
 ('Rv3411c', 'Rv3575c'): 1,
 ('Rv3411c', 'Rv3676'): 0.6416666666666667,
 ('Rv1908c', 'Rv0117'): 1,
 ('Rv1908c', 'Rv1909c'): 0.029411764705882353,
 ('Rv3913', 'Rv0117'): 1,
 ('Rv3913', 'Rv3223c'): 1,
 ('Rv0573c', 'Rv0212c'): 1,
 ('Rv0573c', 'Rv3133c'): 1,
 ('Rv1594', 'Rv0212c'): 1,
 ('Rv1595', 'Rv0212c'): 1,
 ('Rv1596', 'Rv0212c'): 1,
 ('Rv0252', 'Rv0353'): 1,
 ('Rv0252', 'Rv1221'): 1,
 ('Rv0252', 'Rv1785c'): 1,
 ('Rv0252', 'Rv3223c'): 1,
 ('Rv1018c', 'Rv0485'): 1,
 ('Rv1692', 'Rv0485'): 1,
 ('Rv1692', 'Rv3676'): 1,
 ('Rv3332', 'Rv0485'): 1,
 ('Rv3332', 'Rv3676'): 1,
 ('Rv3436c', 'Rv0485'): 1,
 ('Rv0820', 'Rv0491'): 0.5304878048780488,
 ('Rv0936', 'Rv0491'): 1,
 ('Rv0859', 'Rv0586'): 1,
 ('Rv0860', 'Rv0586'): 1,
 ('Rv0524', 'Rv0844c'): 1,
 ('Rv0524', 'Rv2711'): 1,
 ('Rv1029', 'Rv1027c'): 0.5689655172413793,
 ('Rv1030', 'Rv1027c'): 0.9137931034482

In [41]:
# using PROM
prom = PROM(model).build()
solution = prom.optimize(initial_state=initial_state)
solution.solutions

{'ko_Rv0001': PROM Solution
  Objective value: 0.028300772436183185
  Status: optimal,
 'ko_Rv3575c': PROM Solution
  Objective value: 0.05219920249341635
  Status: optimal,
 'ko_Rv3676': PROM Solution
  Objective value: 0.031174348712161924
  Status: optimal,
 'ko_Rv0117': PROM Solution
  Objective value: 0.052199202493360616
  Status: optimal,
 'ko_Rv1909c': PROM Solution
  Objective value: 0.05219920249340569
  Status: optimal,
 'ko_Rv3223c': PROM Solution
  Objective value: 0.05219920249340569
  Status: optimal,
 'ko_Rv0212c': PROM Solution
  Objective value: 0.05219920249340569
  Status: optimal,
 'ko_Rv3133c': PROM Solution
  Objective value: 0.05219920249340569
  Status: optimal,
 'ko_Rv0353': PROM Solution
  Objective value: 0.05219920249340569
  Status: optimal,
 'ko_Rv1221': PROM Solution
  Objective value: 0.05219920249340569
  Status: optimal,
 'ko_Rv1785c': PROM Solution
  Objective value: 0.05219920249340569
  Status: optimal,
 'ko_Rv0485': PROM Solution
  Objective value

In [42]:
# using the slim version. PROM's slim version performs a single KO only. If regulator is None, the first regulator is used.
slim_prom(model, initial_state=initial_state, regulator='Rv0001')

0.028300772436183185

## CoRegFlux
**`CoRegFlux`** is a linear regression-based phenotype simulation method for integrated models. This method circumvents discrete constraints created by **`RFBA`** and **`SRFBA`**. **`CoRegFlux`** uses a continuous approach: reactions' constraints are proportional (using soft plus activation function) to the predicted expression of related genes. This method uses a linear regression model to predict the expression of a target gene as function of the co-expression of its regulators (co-activators and co-repressors). To train a linear regression model, **`CoRegFlux`** uses the target gene expression and regulators' influence scores* from a training dataset. Then, this model is used to make predictions of the target gene expression in the experiment (test) dataset.

*Influence score is a correlation-based score for the activation or repression of a regulator inferred with CoRegNet available at [https://doi.org/10.1093/bioinformatics/btv305](https://doi.org/10.1093/bioinformatics/btv305).

**`CoRegFlux`** performs a single steady-state simulation using the linear regression model predictions to limit flux through some reactions. Hence, this method can predict the phenotypic behavior of an organism in all environmental conditions available in the gene expression dataset. However, this method must use a different training dataset to infer regulators' influence scores and train the linear regression models. **`CoRegFlux`** can also perform dynamic simulations for a series of time steps. At each time step, dynamic **`CoRegFlux`** updates metabolite concentrations and biomass yield using the euler function. These values are then translated into additional constraints to be added to the steady-state simulation.

**`CoRegFlux`** can generate a **`ModelSolution`** containing the solution for a single environmental condition in the experiment dataset. In addition, **`CoRegFlux`** can generate a **`DynamicSolution`** containing time-step solutions for a single environmental condition in the experiment dataset.

**`CoRegFlux`** is available in the **`mewpy.germ.analysis`** package. Alternatively, one can use the simple and optimized version **`slim_coregflux`**.

For more details consult: [https://doi.org/10.1186/s12918-017-0507-0](https://doi.org/10.1186/s12918-017-0507-0).

For this example we will be using the following models and data:
- _S. cerevisae_ iMM904 model available at _models/regulation/iMM904.xml_,
- _S. cerevisae_ TRN inferred with CoRegNet and available at _models/regulation/iMM904_trn.csv_,
- _S. cerevisae_ training gene expression dataset available at _models/regulation/iMM904_gene_expression.csv_,
- _S. cerevisae_ influence scores inferred with CoRegNet in the gene expression dataset available at _models/regulation/iMM904_influence.csv_,
- _S. cerevisae_ experiments gene expression dataset available at _models/regulation/iMM904_experiments.csv_.

In [43]:
# loading model
model = read_model(imm904_gem_reader, imm904_trn_reader)

# objective function
BIOMASS_ID = 'BIOMASS_SC5_notrace'
model.objective = {BIOMASS_ID: 1}
model

Model,iMM904
Name,S. cerevisae iMM904 model - Mo et al 2009
Types,"regulatory, metabolic"
Compartments,"c, e, m, x, r, v, g, n"
Reactions,1577
Metabolites,1226
Genes,905
Exchanges,164
Demands,0
Sinks,0
Objective,BIOMASS_SC5_notrace


**`CoRegFlux`** phenotype simulation requires an initial state that must be inferred from the TRN, gene expression dataset, influence score matrix and experiments gene expression dataset. This initial state contains the predicted gene expression of target metabolic genes available in the GEM model.
<br>
**`mewpy.germ.analysis.coregflux`** module includes the tools to infer **`CoRegFlux`**'s initial state. These methods create the linear regression models to predict targets' expression according to the experiments gene expression dataset. One just have to load expression, influence and experiments CSV files using `mewpy.omics.ExpressionSet`.

HINT: the `predict_gene_expression` method might be time-consuming for some gene expression datasets. One can save the predictions into a CSV file and then load it afterwards using `mewpy.omics.ExpressionSet.from_csv()`.

In [44]:
from mewpy.omics import ExpressionSet

# HINT: you can uncomment the following line to load pre-computed gene expression predictions.
# Do not forget to comment the remaining lines in this cell.
# gene_expression_prediction = ExpressionSet.from_csv(path.joinpath('iMM904_gene_expression_prediction.csv'),
#                                                           sep=',', index_col=0, header=0).dataframe

expression = ExpressionSet.from_csv(path.joinpath('iMM904_gene_expression.csv'), sep=';', index_col=0, header=0).dataframe
influence = ExpressionSet.from_csv(path.joinpath('iMM904_influence.csv'), sep=';', index_col=0, header=0).dataframe
experiments = ExpressionSet.from_csv(path.joinpath('iMM904_experiments.csv'), sep=';', index_col=0, header=0).dataframe

gene_expression_prediction = predict_gene_expression(model=model, influence=influence, expression=expression,
                                                     experiments=experiments)
gene_expression_prediction

,yB8n055.03II01.batch.p1,yB8n056.03II01.batch.p2,yB8n057.03II01.batch.p3,yB8n058.03II01.batch.p4,yB8n059.03II01.batch.p5,yB8n060.03II01.batch.p6,yB8n061.03II01.batch.p7,yB8n062.03II01.batch.p8,yB8n063.03II01.batch.p9,yB8n064.03II01.batch.p10,yB8n065.03II01.batch.p11,yB8n066.03II01.batch.p12,yB8n044.Low.D.chemostat.vs..High.D.chemostat
YMR056C,10.310906,9.374551,9.738155,9.738831,10.383624,9.973555,9.627858,11.579884,11.163626,11.706841,11.655439,9.197184,10.402504
YBR085W,7.999073,7.978798,7.926310,7.744950,7.655623,7.433795,7.883342,6.681351,6.955076,6.468447,6.930435,7.745075,7.469748
YJR155W,8.719261,8.845556,8.779802,8.848888,8.970303,8.862843,8.714360,9.761043,9.762521,9.905502,9.747369,8.684471,9.510130
YDL243C,6.413120,6.802390,6.658224,6.574777,5.948326,6.198785,6.742563,6.209202,6.547471,5.949884,6.172434,6.455557,6.576763
YHR047C,8.230008,8.768411,8.551748,8.697505,8.882710,9.125730,8.896929,8.868654,8.837522,8.688578,8.392062,9.176441,8.538216
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YML002W,7.757591,7.533461,7.617832,7.536168,7.296171,6.948933,7.461951,7.456734,7.535741,7.428243,7.784473,7.058646,7.606172
YML030W,10.530161,10.486551,10.464589,10.667886,10.675506,10.908625,10.757949,11.978087,11.939201,11.908917,11.600359,10.584607,10.869762
YML053C,10.243145,10.102137,10.054621,10.076379,10.088078,10.059266,10.338693,10.617780,10.593350,10.525567,10.571640,9.983406,10.491541
YML087C,6.589562,6.591232,6.598206,6.712599,6.727422,6.741082,6.576150,7.834831,7.822760,7.842013,7.334910,6.708636,6.898609


In [45]:
# steady-state simulation only requires the initial state of a given experiment (the first experiment in this case)
initial_state = list(gene_expression_prediction.to_dict().values())
co_reg_flux = CoRegFlux(model).build()
solution = co_reg_flux.optimize(initial_state=initial_state[0])
solution

Method,CoRegFlux
Model,Model iMM904 - S. cerevisae iMM904 model - Mo et al 2009
Objective,BIOMASS_SC5_notrace
Objective value,0.2878657037040182
Status,optimal


In [46]:
# using the simple version of CoRegFlux
slim_coregflux(model, initial_state=initial_state[0])

0.2878657037040182

In [47]:
# dynamic simulation requires metabolite concentrations, wt growth rate and initial state
metabolites = {'glc__D_e': 16.6, 'etoh_e': 0}
growth_rate = 0.45
time_steps = list(range(1, 14))

co_reg_flux = CoRegFlux(model).build()
solution = co_reg_flux.optimize(initial_state=initial_state,
                                metabolites=metabolites,
                                growth_rate=growth_rate,
                                time_steps=time_steps)
solution.solutions

{'t_1': CoRegFlux Solution
  Objective value: 0.28786570367625985
  Status: optimal,
 't_2': CoRegFlux Solution
  Objective value: 0.287865703704015
  Status: optimal,
 't_3': CoRegFlux Solution
  Objective value: 0.181837987723084
  Status: optimal,
 't_4': CoRegFlux Solution
  Objective value: 0.02916276694278548
  Status: optimal,
 't_5': CoRegFlux Solution
  Objective value: 0.029162766956663245
  Status: optimal,
 't_6': CoRegFlux Solution
  Objective value: 0.029162766956663245
  Status: optimal,
 't_7': CoRegFlux Solution
  Objective value: 0.029162766956663245
  Status: optimal,
 't_8': CoRegFlux Solution
  Objective value: 0.029162766956663245
  Status: optimal,
 't_9': CoRegFlux Solution
  Objective value: 0.029162766956663245
  Status: optimal,
 't_10': CoRegFlux Solution
  Objective value: 0.029162766956663245
  Status: optimal,
 't_11': CoRegFlux Solution
  Objective value: 0.029162766956663245
  Status: optimal,
 't_12': CoRegFlux Solution
  Objective value: 0.02916276694